<a href="https://colab.research.google.com/github/DamienShahan/Klassifikationsverfahren-in-Python/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Klassifikationsverfahren in Python: Modellentwicklung zur Kreditvorhersage einer Bank

# Inhaltsverzeichnis

1. Business Understanding
2. Data Understanding
3. Data Preperation
4. Modelling
5. Evalutation
6. Werte predicten
7. Deployment

# 1. Business Understanding

<b>Wie ist die Ausgangssituation?</b>

Eine Bank möchte für die Zukunft bessere Vorhersagen treffen können, ob ein Kunde seinen Kredit zurückzahlen wird oder nicht.

<b>Welche Ressourcen stehen zur Verfügung?</b>

Zur Entwicklung eines Modells, wurde ein Datensatz mit 120000 Kundeneinträgen zur Verfügung gestellt anhand der das Modell trainiert werden soll. Dazu haben wir von einem Verantwortlichen der Bank eine kurze Beschreibung der Daten in Form eines Data Dictionaries.

<b>Welche Ergebnisse sollen erreicht werden?</b>

Es soll Modell für eine Bank entwickelt werden, welches vohersagen soll, ob ein Kunde seinen Kredit zurückzahlen wird oder nicht. Dazu wird eine komplette Analyse nach dem CRISP-DM Referenzmodell durchgeführt. Zum Schluss wird das Trainingsset auf ein Testset angewandt und versucht deren Zielattribut zu prognostizieren.

# 2. Data Understanding
Das Data Understanding beinhaltet eine deskriptive und eine grafische Analyse. In der deskritiven Analyse werden die wichtigste deskriptiven Maßzahlen für die Daten erhoben und interpretiert. In der grafischen Analyse werden Diagramm erstellt, die die wichtigsten Maßzahlen visualisieren.

Im folgenden haben wir je Spalte in dem gegebenen Trainingsset sowohl eine deskriptive als auch eine grafische Analyse erstellt. Bevor mit dem Data Understanding begonnen werden kann, installieren wir alle benötigten Pakete und laden diese.

Anschließend wird das Trainingsset in einen Dataframe (df) geladen und ausgegeben. Der Dezimaltrenner (decimal) des Datensatzes ist ein Punkt und der Spaltentrenner (seperator) ein Komma.